<a href="https://colab.research.google.com/github/VenkyGitRep/AdvancedPerception-ColorConstancy/blob/main/CC_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import cv2
import numpy as np
import os

In [17]:
#Linearises image, returns an image with values in [-1,1].
#Black level is 2048(as in IPG page), saturation level 15k
'''
Image: /content/drive/MyDrive/ColorConstancy/RawImages/CR2_1_100/3.CR2
Black Level per channel: [2048, 2049, 2049, 2049]
Saturation Level: 15831
'''
#Linearize images
def linearize(img, black_lvl=2048, saturation_lvl=15831):

    linearized_img = np.clip((img - black_lvl)/(saturation_lvl - black_lvl), 0, 1)
    scaled_img = (linearized_img * 2) - 1
    return scaled_img


**Resize all images to fit MobileNet input size of (224,224,3)**

In [18]:
def preprocessImage(img):
  return cv2.resize(linearize(png_img), (224, 224), interpolation=cv2.INTER_LINEAR)


**Linearize all png images**

In [36]:
#preprocessImagesInDirectory
def preprocessImagesInDirectory(image_directory):
  resized_linear_images = []
  # List of image paths
  image_paths = [os.path.join(image_directory, file) for file in os.listdir(image_directory) if file.endswith(('.PNG'))]

  for i in range(len(image_paths)):
      raw_img= cv2.imread(image_paths[i], cv2.IMREAD_UNCHANGED)
      resized_linear_images.append(preprocessImage(raw_img))
      #if i==100:
        #break
      print(i,end=' ')
      if(i%20==0):
        print()
      #cv2.imwrite(f"/content/drive/MyDrive/ColorConstancy/LinearizedRawImages/{i}.png", linearized_image)
  return resized_linear_images

resized_linear_images = preprocessImagesInDirectory("/content/drive/MyDrive/ColorConstancy/CubePlus200Imgs/")


0 
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 
141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 
181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 
221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 
241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 
261 262 263 264 265 266 267 268 269 270 271 272 273 

In [37]:
print(resized_linear_images[0].shape)

(224, 224, 3)


**Save preprocesses images**

In [38]:
# Save the linearized image as a .npy file

def saveNPArray(filename,array):
  # Define the directory where you want to save the file
  save_directory = "/content/drive/MyDrive/ColorConstancy/PreprocessesImages/"

  # Check if the directory exists, if not, create it
  if not os.path.exists(save_directory):
      os.makedirs(save_directory)

  # Define the full path for the .npy file
  file_path = os.path.join(save_directory, filename)

  # Save the linearized image as a .npy file in the specified directory
  np.save(file_path, array)

saveNPArray("linearized_raw_image_nparray.npy",resized_linear_images)

In [39]:
# Later, you can load the image back from that directory
loaded_linearised_images = np.load("/content/drive/MyDrive/ColorConstancy/PreprocessesImages/linearized_raw_image_nparray.npy")

In [40]:
# Compute the difference
difference = np.abs(resized_linear_images - loaded_linearised_images)
print(difference.max())

0.0


**Load ground truth values: Illumination values for PNG image**

In [41]:
#Getting grount truth illumination values
file_path = '/content/drive/MyDrive/ColorConstancy/cube+_gt.txt'

illuminant_true_values = []

with open(file_path, 'r') as file:
    for i, line in enumerate(file):
      values = line.strip().split()
      if len(values) == 3:
            illuminant_true_values.append([float(value) for value in values])


In [42]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import layers,models
import tensorflow as tf


# Define a custom regression model based on MobileNetV2
def create_model():
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = False  # Freeze the pretrained layers

    # Add custom layers for regression
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(128, activation='relu')(x)
    output = layers.Dense(3, activation='linear')(x)  # Output layer with 3 units for RGB illuminant values

    model = models.Model(inputs=base_model.input, outputs=output)

    return model

# Create the model
model = create_model()

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Create a data generator using tf.data.Dataset
batch_size = 32  # Adjust batch size as needed
# Convert the lists to TensorFlow tensors
linearised_png_images_tensor = tf.convert_to_tensor(loaded_linearised_images, dtype=tf.float32)
illuminant_true_values_tensor = tf.convert_to_tensor(illuminant_true_values[0:len(loaded_linearised_images)], dtype=tf.float32)

# Create a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((linearised_png_images_tensor, illuminant_true_values_tensor))

# Optionally, you can add dataset operations such as batching, shuffling, etc.
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size=32)

# Train the model
num_epochs = 100  # Adjust the number of training epochs
history = model.fit(train_dataset, epochs=num_epochs, verbose=1)

Epoch 1/100
19/19 [==============================] - 3s 18ms/step - loss: 0.5351
Epoch 2/100
19/19 [==============================] - 0s 16ms/step - loss: 0.0547
Epoch 3/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0121
Epoch 4/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0042
Epoch 5/100
19/19 [==============================] - 0s 16ms/step - loss: 0.0034
Epoch 6/100
19/19 [==============================] - 0s 16ms/step - loss: 0.0034
Epoch 7/100
19/19 [==============================] - 0s 16ms/step - loss: 0.0036
Epoch 8/100
19/19 [==============================] - 0s 16ms/step - loss: 0.0038
Epoch 9/100
19/19 [==============================] - 0s 16ms/step - loss: 0.0049
Epoch 10/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0042
Epoch 11/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0036
Epoch 12/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0043
Epoch 13/100
19/19 [=====

**Make a preiction**


In [43]:
test_png_imag_path="/content/drive/MyDrive/ColorConstancy/TestImages/1001.PNG"

#Perform preprocessing:
png_img=cv2.imread(test_png_imag_path, cv2.IMREAD_UNCHANGED)
resized_linear_image = preprocessImage(png_img)

In [44]:

# Make predictions

# Expand dimensions to match the shape of (batch_size, height, width, channels)
img_batch = np.expand_dims(resized_linear_image, axis=0)

# Predict the values using the trained model
predictions = model.predict(img_batch)

predictions


1/1 [==============================] - 1s 769ms/step


array([[0.2083752 , 0.4479513 , 0.33874932]], dtype=float32)

1001.PNG GT       :[0.23895582 0.47590361 0.28514056]

First prediction  : [0.46585822, 0.7702673 , 0.03647143]

Second prediction : [0.2062704 , 0.24868996, 0.08920331]

Third prediction  : [0.20485055, 0.4598795 , 0.3357277 ]

**Preprocess test images**

In [45]:
linearized_test_images = preprocessImagesInDirectory("/content/drive/MyDrive/ColorConstancy/TestImages")
saveNPArray("linearized_test_images_nparray.npy",linearized_test_images)

0 
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 
141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 
181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 

**Predictions made with 200 test images**

In [47]:
# Create a data generator for inference
inference_dataset = tf.data.Dataset.from_tensor_slices(linearized_test_images)
inference_dataset = inference_dataset.batch(batch_size)

# Make predictions
predictions= model.predict(inference_dataset)
predictions.shape


7/7 [==============================] - 1s 22ms/step


(200, 3)

**Metrics**

In [48]:
def angularError(true_illumination, prediction):
  # Normalize the vectors
    true_color_normalized = true_illumination / np.linalg.norm(true_illumination)
    estimated_color_normalized = prediction / np.linalg.norm(prediction)

    # Compute the dot product
    dot_product = np.dot(true_color_normalized, estimated_color_normalized)

    # Clip the dot product to avoid numerical issues with arccos
    dot_product = np.clip(dot_product, -1.0, 1.0)

    # Compute the angular error in radians
    angle_rad = np.arccos(dot_product)

    # Convert the angular error to degrees
    angle_deg = np.degrees(angle_rad)

    return angle_deg




In [49]:
angularError(illuminant_true_values[1001],predictions[0])

1.9478522226862471

**Summary statistics**

In [50]:
import statistics

def summary_statistics(angular_errors):
    # Step 1: Sort the array
    sorted_arr = np.sort(angular_errors)

    # Step 2: Calculate Q1 (25th percentile)
    q1 = np.percentile(sorted_arr, 25)

    # Step 3: Calculate Q3 (75th percentile)
    q3 = np.percentile(sorted_arr, 75)

    # Step 4: Calculate the median (50th percentile)
    median = statistics.median(sorted_arr)

    # Step 5: Calculate the trimean
    trimean = (q1 + 2 * median + q3) / 4

    # Step 6: Calculate the average of the lowest 25% of values
    lower_25_avg = np.mean(sorted_arr[:len(sorted_arr)//4])

    # Step 7: Calculate the average of the highest 25% of values
    upper_25_avg = np.mean(sorted_arr[-(len(sorted_arr)//4):])

    # Step 8: Calculate the overall mean
    overall_mean = np.mean(sorted_arr)

    # Print the results
    print("Mean:", overall_mean,end='\n')
    print("Median:", median,end='\n')
    print("Trimean:", trimean,end='\n')
    print("Average of the Lowest 25%:", lower_25_avg,end='\n')
    print("Average of the Highest 25%:", upper_25_avg,end='\n')


In [62]:
type(illuminant_true_values[1001:1201])

list

In [61]:
type(predictions)

numpy.ndarray

In [64]:
# Compute the angular error for each pair of true and predicted illuminations
angular_errors = [angularError(true_ill, pred_ill) for true_ill, pred_ill in zip(np.array(illuminant_true_values[1001:1201]), predictions)]
summary_statistics(angular_errors)

Mean: 6.328763261060834
Median: 2.5137465019055325
Trimean: 3.055577905310744
Average of the Lowest 25%: 1.8975095961435278
Average of the Highest 25%: 17.895648026718423


**Compile model with Angular error**

In [75]:
def angular_error_loss(y_true, y_pred):
    y_true_norm = tf.nn.l2_normalize(y_true, axis=1)
    y_pred_norm = tf.nn.l2_normalize(y_pred, axis=1)
    dot_product = tf.reduce_sum(tf.multiply(y_true_norm, y_pred_norm), axis=1)
    dot_product = tf.clip_by_value(dot_product, -1.0, 1.0)

    dot_product_checked = tf.debugging.check_numerics(dot_product, "dot_product")
    angle_rad = tf.acos(dot_product_checked)


    angle_rad = tf.acos(dot_product)
    angle_deg = tf.math.scalar_mul(tf.constant(180.0 / np.pi, dtype=tf.float32), angle_rad)
    return angle_deg

In [79]:

# Create the model
model = create_model()

# Compile the model
model.compile(optimizer='adam', loss='mse')

In [80]:
# Train the model
num_epochs = 100  # Adjust the number of training epochs
history = model.fit(train_dataset, epochs=num_epochs, verbose=1)

Epoch 1/100
19/19 [==============================] - 3s 18ms/step - loss: 0.6207
Epoch 2/100
19/19 [==============================] - 0s 18ms/step - loss: 0.0579
Epoch 3/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0107
Epoch 4/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0052
Epoch 5/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0045
Epoch 6/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0047
Epoch 7/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0044
Epoch 8/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0042
Epoch 9/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0036
Epoch 10/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0036
Epoch 11/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0033
Epoch 12/100
19/19 [==============================] - 0s 17ms/step - loss: 0.0034
Epoch 13/100
19/19 [=====

In [81]:
predictions

array([[0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.2083746 , 0.44795197, 0.33874905],
       [0.